In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google'

In [45]:
import pandas as pd
import numpy as np
import tensorflow 
from sklearn.metrics import accuracy_score, precision_score, recall_score,confusion_matrix, fbeta_score, precision_recall_curve,average_precision_score
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Conv1D, MaxPool1D, Dense, Dropout, Flatten,BatchNormalization, Input, concatenate, Activation
from tensorflow.keras.optimizers import Adam
from keras.models import model_from_json
import warnings

warnings.filterwarnings('ignore')
from inspect import signature
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
data = pd.read_csv('/content/gdrive/My Drive/file1.csv')

In [ ]:
data.head()

,Unnamed: 0,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,target
0,index_0,0.0,4.6,8.0,0.0,0.0,0.3,0.2,0.0,0.1,0.0,0
1,index_1,0.0,0.0,0.0,0.5,1.5,2.7,5.4,0.5,0.1,8.7,0
2,index_2,5.1,2.4,2.1,0.3,3.8,1.8,22.2,24.3,0.4,0.0,1
3,index_3,0.0,0.0,5.5,0.8,6.1,0.7,15.3,36.5,2.2,2.0,1
4,index_4,1.8,29.2,2.8,7.6,11.8,4.1,5.7,1.1,6.3,1.3,1


In [48]:
data.shape

(907, 12)

In [49]:
train_y = data.target[:700]
train_x = data[:700] 

In [50]:
test_y = data.target[700:]
test_x = data[700:] 

In [51]:
print(train_y.shape)
Y_train = train_y

(700,)


In [52]:
print(train_x.iloc[:,1:11].shape)
X_train = train_x.iloc[:,1:11]

(700, 10)


In [53]:
print(test_y.shape)
Y_test = test_y

(207,)


In [54]:
print(test_x.iloc[:,1:11].shape)
X_test = test_x.iloc[:,1:11]

(207, 10)


In [55]:
x_train = np.stack([X_train, X_train], axis=2)
x_test = np.stack([X_test, X_test], axis=2)

In [63]:
model = Sequential()
model.add(Conv1D(filters=8, kernel_size=4, activation='linear', input_shape=x_train.shape[1:]))
model.add(MaxPool1D(strides=2))
model.add(BatchNormalization())
model.add(Dense(20, activation='relu'))
model.add(MaxPool1D(strides=2))
model.add(Flatten())
model.add(Dropout(0.25)) 
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [65]:
model.compile(optimizer=Adam(4e-5), loss = 'binary_crossentropy', metrics=['accuracy'])
hist = model.fit(x_train, Y_train,validation_data=(x_test, Y_test),verbose=2, epochs=100,steps_per_epoch=50)

Epoch 1/100
50/50 - 1s - loss: 0.6838 - accuracy: 0.7443 - val_loss: 0.6618 - val_accuracy: 0.8696 - 1s/epoch - 22ms/step
Epoch 2/100
50/50 - 0s - loss: 0.6817 - accuracy: 0.7457 - val_loss: 0.6608 - val_accuracy: 0.8696 - 96ms/epoch - 2ms/step
Epoch 3/100
50/50 - 0s - loss: 0.6694 - accuracy: 0.7757 - val_loss: 0.6599 - val_accuracy: 0.8647 - 113ms/epoch - 2ms/step
Epoch 4/100
50/50 - 0s - loss: 0.6740 - accuracy: 0.7600 - val_loss: 0.6590 - val_accuracy: 0.8647 - 118ms/epoch - 2ms/step
Epoch 5/100
50/50 - 0s - loss: 0.6716 - accuracy: 0.7757 - val_loss: 0.6581 - val_accuracy: 0.8696 - 114ms/epoch - 2ms/step
Epoch 6/100
50/50 - 0s - loss: 0.6739 - accuracy: 0.7700 - val_loss: 0.6573 - val_accuracy: 0.8647 - 114ms/epoch - 2ms/step
Epoch 7/100
50/50 - 0s - loss: 0.6738 - accuracy: 0.7714 - val_loss: 0.6565 - val_accuracy: 0.8841 - 98ms/epoch - 2ms/step
Epoch 8/100
50/50 - 0s - loss: 0.6674 - accuracy: 0.7957 - val_loss: 0.6558 - val_accuracy: 0.8889 - 95ms/epoch - 2ms/step
Epoch 9/100
5

In [66]:
model_json = model.to_json()
with open("/content/gdrive/My Drive/CNNmodel.json", "w") as  json_file:
  json_file.write(model_json)
model.save_weights("/content/gdrive/My Drive/CNNmodel.h5")
print("Saved model to disk")

Saved model to disk


In [67]:
json_file = open('/content/gdrive/My Drive/CNNmodel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("/content/gdrive/My Drive/CNNmodel.h5") 
print("Loaded model from disk")  
loaded_model.summary()

Loaded model from disk
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_20 (Conv1D)          (None, 7, 8)              72        
                                                                 
 max_pooling1d_14 (MaxPoolin  (None, 3, 8)             0         
 g1D)                                                            
                                                                 
 batch_normalization_8 (Batc  (None, 3, 8)             32        
 hNormalization)                                                 
                                                                 
 dense_9 (Dense)             (None, 3, 20)             180       
                                                                 
 max_pooling1d_15 (MaxPoolin  (None, 1, 20)            0         
 g1D)                                                            
                              

In [68]:
y_pred = model.predict(x_test)[:,0]

In [85]:
pred = np.empty((1,len(y_pred)), dtype=object)
pred = np.where(y_pred>=0.47, 1, 0)
y_train = np.reshape(Y_train,len(Y_train))
pred = np.reshape(pred,len(pred))

In [86]:
pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0])